<a href="https://colab.research.google.com/github/apanand/UChicago-MSADS/blob/main/Spotify_Recommender_WITH_MY_STREAMING_DATA_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from numpy import argmax
import pandas as pd
from collections import Counter
from datetime import datetime
import os
import itertools
import math
import collections

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# !pip install ipyplot
# !pip install jupyter-dash
import seaborn as sns
import matplotlib.pyplot as plt

# ## Neural Network
# #!pip install tensorflow
# import tensorflow as tf
# from tensorflow.keras.activations import relu, sigmoid, softmax, tanh, selu, elu
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
# from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# !pip install tensorflow_addons
# from tensorflow.keras.constraints import MaxNorm
# from tensorflow.keras.regularizers import l2, l1
# from tensorboard.plugins.hparams import api as hp
# !pip install keras-tuner
# import kerastuner as kt
# !pip install tensorflow_addons
# import tensorflow_addons as tfa
# !pip install scikeras[tensorflow]
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# import keras_tuner

# Spotify
! pip3 install spotipy
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth


# SkLearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 6.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving StreamingHistory_music_0.json to StreamingHistory_music_0.json
Saving StreamingHistory_music_1.json to StreamingHistory_music_1.json
Saving StreamingHistory_music_2.json to StreamingHistory_music_2.json
Saving StreamingHistory_music_3.json to StreamingHistory_music_3.json


In [ ]:
import pandas as pd
import json

def load_and_merge_json_files(file_paths):
    # Initialize an empty list to hold DataFrames
    dataframes = []

    # Loop through each file path and read the JSON file into a DataFrame
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            df = pd.DataFrame(data)
            dataframes.append(df)

    # Concatenate all DataFrames into a single DataFrame
    merged_df = pd.concat(dataframes, ignore_index=True)

    return merged_df

##Example usage:
file_paths = ['StreamingHistory_music_0.json', 'StreamingHistory_music_1.json', 'StreamingHistory_music_2.json', 'StreamingHistory_music_3.json']
merged_df = load_and_merge_json_files(file_paths)
print(merged_df)

                endTime      artistName  \
0      2023-05-18 00:02     JACK MARLOW   
1      2023-05-18 00:03         Kartoon   
2      2023-05-18 02:04          salute   
3      2023-05-18 03:05    TECH IT DEEP   
4      2023-05-18 03:10  Shaolin Cowboy   
...                 ...             ...   
33437  2024-05-18 23:56    Flipp Dinero   
33438  2024-05-18 23:56          Future   
33439  2024-05-18 23:56    Lil Uzi Vert   
33440  2024-05-18 23:56         J. Cole   
33441  2024-05-18 23:57           Gunna   

                                       trackName  msPlayed  
0                                     Flyin Baby    192000  
1      Soundboy Surrender - Crossy Remix - Mixed     61714  
2                                          Peach    345310  
3                                    Maria Maria     31151  
4                        Réveiller (feat. manon)    289787  
...                                          ...       ...  
33437                             Leave Me Alone      32

In [ ]:
merged_df

,endTime,artistName,trackName,msPlayed
0,2023-05-18 00:02,JACK MARLOW,Flyin Baby,192000
1,2023-05-18 00:03,Kartoon,Soundboy Surrender - Crossy Remix - Mixed,61714
2,2023-05-18 02:04,salute,Peach,345310
3,2023-05-18 03:05,TECH IT DEEP,Maria Maria,31151
4,2023-05-18 03:10,Shaolin Cowboy,Réveiller (feat. manon),289787
...,...,...,...,...
33437,2024-05-18 23:56,Flipp Dinero,Leave Me Alone,3200
33438,2024-05-18 23:56,Future,Mask Off (feat. Kendrick Lamar) - Remix,5116
33439,2024-05-18 23:56,Lil Uzi Vert,New Patek,3480
33440,2024-05-18 23:56,J. Cole,MIDDLE CHILD,1200


In [ ]:
merged_df.value_counts()

endTime           artistName          trackName                        msPlayed
2023-05-18 00:02  JACK MARLOW         Flyin Baby                       192000      1
2024-02-28 18:48  Magic City Hippies  Ghost On The Mend - veggi remix  88526       1
2024-02-28 16:07  Unknown T           Goodums - Sammy Virji Remix      9088        1
                  Sammy Virji         Find My Way Home                 6816        1
                  RAC                 Passion                          3455        1
                                                                                  ..
2023-10-24 19:06  Oppidan             Rosalina                         169411      1
2023-10-24 19:03  JEV                 DREAMER                          133846      1
2023-10-24 19:01  MPH                 Spend The Night                  210909      1
2023-10-24 18:58  Conducta            Alone - Mixed                    112923      1
2024-05-18 23:57  Gunna               Drip or Drown                   

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33442 entries, 0 to 33441
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   endTime     33442 non-null  object
 1   artistName  33442 non-null  object
 2   trackName   33442 non-null  object
 3   msPlayed    33442 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [ ]:
merged_df = merged_df.drop_duplicates(subset=['trackName', 'artistName'])
merged_df


,endTime,artistName,trackName,msPlayed
0,2023-05-18 00:02,JACK MARLOW,Flyin Baby,192000
1,2023-05-18 00:03,Kartoon,Soundboy Surrender - Crossy Remix - Mixed,61714
2,2023-05-18 02:04,salute,Peach,345310
3,2023-05-18 03:05,TECH IT DEEP,Maria Maria,31151
4,2023-05-18 03:10,Shaolin Cowboy,Réveiller (feat. manon),289787
...,...,...,...,...
33416,2024-05-18 23:52,m$sha,OOPS,900
33418,2024-05-18 23:52,CALIsthenics,"Paper Thick (feat. E-40, Casual & Del the Funk...",1330
33421,2024-05-18 23:52,Chris Mula,Go To Work,620
33428,2024-05-18 23:56,Travis Scott,Oh My Dis Side (feat. Quavo),940


In [6]:
#@title Spotify Developer Credentials { run: "auto", vertical-output: true, display-mode: "form" }
SPOTIFY_CLIENT_ID = "63491f3b28294698834a8596f0ae0a22" #@param {type:"string"}
SPOTIFY_CLIENT_SECRET = "9c708402905d434d83b87657eef5d30b" #@param {type:"string"}
SPOTIFY_REDIRECT_URI = 'https://www.google.com/'
SCOPE = "playlist-modify-public playlist-read-collaborative playlist-modify-private"
scope = "user-read-recently-played"
assert len(SPOTIFY_CLIENT_ID)>0, 'Please provide a spotify client id'
assert len(SPOTIFY_CLIENT_SECRET)>0, 'Please provide a spotify client secret code'

sp = spotipy.Spotify(
    auth_manager=spotipy.SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=SPOTIFY_REDIRECT_URI,
        scope=SCOPE, open_browser=False),
        requests_timeout=20, retries=10)
form_conn = sp.artist('spotify:artist:3jOstUTkEu2JkjvRdBA5Gu')

print('Authorization Sucessful!')

Go to the following URL: https://accounts.spotify.com/authorize?client_id=63491f3b28294698834a8596f0ae0a22&response_type=code&redirect_uri=https%3A%2F%2Fwww.google.com%2F&scope=playlist-modify-public+playlist-read-collaborative+playlist-modify-private
Enter the URL you were redirected to: https://www.google.com/?code=AQAWvaTURHykdtQ68Kb-ZSLshhe85Dpdu1f1zPEjlgsrQEEo97R07haK-mww0DYFvwCAXhiG-DsxV6eICdbtbO2oy2KMI3k_jt8CjHiOhontsXuop2HXwnAvKCB6v7m3e2KD5Hxmw6zgEwTvuZZxEgf2PE4nkosevQ6a3bAfowpe7sZ5Zw4rNKvjO30KGEXNdbWoOoY0lvVSPnWfQ03Qkld5SxBhj8tAsaH9od2IFeiVRsGIbClUJAB25ryFELPfcmo-FkQLwazd
Authorization Sucessful!


In [ ]:
import time
from tqdm import tqdm

def get_track_id(artist_name, track_name):
    query = f"artist:{artist_name} track:{track_name}"
    while True:
        try:
            result = sp.search(q=query, type='track', limit=1)
            tracks = result.get('tracks', {}).get('items', [])
            if tracks:
                return tracks[0]['id']
            return None
        except spotipy.SpotifyException as e:
            if e.http_status == 429:  # Rate limit status code
                retry_after = int(e.headers.get('Retry-After', 60))
                print(f"Rate limit exceeded. Sleeping for {retry_after} seconds.")
                time.sleep(retry_after)
            else:
                print(f"SpotifyException: {e}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"RequestException: {e}")
            time.sleep(5)  # Wait for a short time before retrying
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None

# Assuming your DataFrame is named `df`
# Initialize an empty list to store track IDs
track_ids = []

# Loop through the DataFrame with a progress bar and handle rate limits
for _, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Fetching track IDs"):
    track_id = get_track_id(row['artistName'], row['trackName'])
    track_ids.append(track_id)
    time.sleep(0.5)  # Add a delay to avoid hitting the rate limit too quickly

# Assign the track IDs back to the DataFrame
merged_df['track_id'] = track_ids


# def get_track_id(artist_name, track_name):
#     query = f"artist:{artist_name} track:{track_name}"
#     result = sp.search(q=query, type='track', limit=1)
#     tracks = result.get('tracks', {}).get('items', [])
#     if tracks:
#         return tracks[0]['id']
#     return None

# # Apply the get_track_id function with a progress bar
# merged_df['track_id'] = [get_track_id(row['artistName'], row['trackName']) for _, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Fetching track IDs")]

# time.sleep(1)


Fetching track IDs:   5%|▍         | 387/8380 [04:48<1:39:21,  1.34it/s]


KeyboardInterrupt: 

In [ ]:
merged_df.to_csv('merged_df.csv', index=False)

from google.colab import files
files.download("merged_df.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from google.colab import files

uploaded = files.upload()

Saving merged_df.csv to merged_df.csv


In [7]:
merged_df = pd.read_csv('merged_df.csv')

In [8]:
merged_df

,endTime,artistName,trackName,msPlayed,track_id
0,2023-05-18 00:02,JACK MARLOW,Flyin Baby,192000,597vP6rhNkEpSHpCyriFnf
1,2023-05-18 00:03,Kartoon,Soundboy Surrender - Crossy Remix - Mixed,61714,3cEvwcgCxyMKIKFp37LGwS
2,2023-05-18 02:04,salute,Peach,345310,5cGZN0P1QnSfhCFBCHtp2N
3,2023-05-18 03:05,TECH IT DEEP,Maria Maria,31151,4wP6mzWZC94dYmU6OYeljf
4,2023-05-18 03:10,Shaolin Cowboy,Réveiller (feat. manon),289787,51oib6a2dRu2f6x8OMMphQ
...,...,...,...,...,...
8375,2024-05-18 23:52,m$sha,OOPS,900,4UBIs1W680L7JR7oqkx3zq
8376,2024-05-18 23:52,CALIsthenics,"Paper Thick (feat. E-40, Casual & Del the Funk...",1330,4uSsR7XGqmC4F8ngY65I2M
8377,2024-05-18 23:52,Chris Mula,Go To Work,620,3AwQtBfYQgDLfMnQ8ZNkEt
8378,2024-05-18 23:56,Travis Scott,Oh My Dis Side (feat. Quavo),940,2rMFawCg4BW65jzbwztXAV


In [9]:
track_ids = merged_df['track_id']

In [10]:
track_ids

0       597vP6rhNkEpSHpCyriFnf
1       3cEvwcgCxyMKIKFp37LGwS
2       5cGZN0P1QnSfhCFBCHtp2N
3       4wP6mzWZC94dYmU6OYeljf
4       51oib6a2dRu2f6x8OMMphQ
                 ...          
8375    4UBIs1W680L7JR7oqkx3zq
8376    4uSsR7XGqmC4F8ngY65I2M
8377    3AwQtBfYQgDLfMnQ8ZNkEt
8378    2rMFawCg4BW65jzbwztXAV
8379    6nO3tr47nr2P7f3hXb8JIo
Name: track_id, Length: 8380, dtype: object

In [14]:
import sys
from tqdm import tqdm



def get_track_details_and_features(sp, track_ids):
    details_list = []

    for i in tqdm(range(0, len(track_ids), 50), desc="Fetching track details and features"):
        batch = track_ids[i:i + 50]
        retries = 0
        while True:
            try:
                # Fetch track details
                tracks = sp.tracks(batch)['tracks']
                # Fetch track features
                features_list = sp.audio_features(batch)

                # Combine track details and features
                for track, features in zip(tracks, features_list):
                    if track and features:  # Ensure there are track details and features
                        details = {
                            'id': track['id'],
                            'title': track['name'],
                            'all_artists': ', '.join([artist['name'] for artist in track['artists']]),
                            'popularity': track['popularity'],
                            'release_date': track['album']['release_date'],
                            'danceability': features['danceability'],
                            'energy': features['energy'],
                            'key': features['key'],
                            'loudness': features['loudness'],
                            'mode': features['mode'],
                            'acousticness': features['acousticness'],
                            'instrumentalness': features['instrumentalness'],
                            'liveness': features['liveness'],
                            'valence': features['valence'],
                            'tempo': features['tempo'],
                            'duration_ms': features['duration_ms'],
                            'time_signature': features['time_signature']
                        }
                        details_list.append(details)
                break  # Exit the retry loop if the call is successful
            except spotipy.SpotifyException as e:
                if e.http_status == 429:  # Rate limit status code
                    retries += 1
                    retry_after = min(int(e.headers.get('Retry-After', 60)), 2 ** retries)
                    print(f"Rate limit exceeded. Sleeping for {retry_after} seconds. Retry #{retries}", file=sys.stderr)
                    time.sleep(retry_after)
                else:
                    print(f"SpotifyException: {e}", file=sys.stderr)
                    break
            except requests.exceptions.RequestException as e:
                print(f"RequestException: {e}", file=sys.stderr)
                time.sleep(5)  # Wait for a short time before retrying
            except Exception as e:
                print(f"Unexpected error: {e}", file=sys.stderr)
                break
    return details_list

In [15]:
track_ids = merged_df['track_id'].dropna().tolist()

# Fetch details and features
track_details = get_track_details_and_features(sp, track_ids)

# Convert the list of details into a DataFrame
details_df = pd.DataFrame(track_details)


Fetching track details and features: 100%|██████████| 163/163 [01:09<00:00,  2.33it/s]


In [21]:
details_df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,597vP6rhNkEpSHpCyriFnf,Flyin Baby,JACK MARLOW,39,2023-04-21,0.826,0.576,9,-10.317,0,0.0165,0.747000,0.1340,0.670,129.980,192000,4
1,3cEvwcgCxyMKIKFp37LGwS,Soundboy Surrender - Crossy Remix - Mixed,"Kartoon, Crossy",55,2022-07-22,0.530,0.856,10,-2.220,0,0.0176,0.351000,0.2450,0.718,87.513,61714,4
2,5cGZN0P1QnSfhCFBCHtp2N,Peach,"salute, Sammy Virji",55,2023-03-31,0.866,0.668,1,-7.472,1,0.0115,0.889000,0.2380,0.657,135.009,345311,4
3,4wP6mzWZC94dYmU6OYeljf,Maria Maria,TECH IT DEEP,69,2023-02-08,0.723,0.685,0,-6.790,1,0.0992,0.000601,0.2440,0.386,127.987,161802,4
4,51oib6a2dRu2f6x8OMMphQ,Réveiller (feat. manon),"Shaolin Cowboy, manon",52,2021-03-26,0.842,0.418,8,-9.602,0,0.7650,0.022800,0.1170,0.448,128.009,289787,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8095,4UBIs1W680L7JR7oqkx3zq,OOPS,"m$sha, Jay Critch",42,2023-03-17,0.790,0.652,3,-6.882,0,0.0325,0.000041,0.0769,0.253,140.399,117447,4
8096,4uSsR7XGqmC4F8ngY65I2M,"Paper Thick (feat. E-40, Casual & Del the Funk...","CALIsthenics, E-40, Casual, Del The Funky Homo...",32,2024-03-29,0.770,0.770,11,-6.348,0,0.0177,0.005410,0.3220,0.261,104.105,124824,4
8097,3AwQtBfYQgDLfMnQ8ZNkEt,Go To Work,Chris Mula,66,2023-10-17,0.509,0.540,9,-13.219,1,0.5210,0.000000,0.4570,0.152,67.770,116286,4
8098,2rMFawCg4BW65jzbwztXAV,Oh My Dis Side (feat. Quavo),"Travis Scott, Quavo",66,2015-09-04,0.459,0.623,1,-4.450,1,0.3520,0.000000,0.1310,0.205,126.014,351253,4


In [19]:
details_df.to_csv('listening_history.csv', index=False)

from google.colab import files
files.download("listening_history.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def get_track_features(track_ids):
    features_list = []
    index = 0

    # Process in batches of 100 tracks due to API limitations
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        features = sp.audio_features(batch)

        for track, feature in zip(batch, features):
            if feature:  # Ensure the feature is not None
                track_data = sp.track(track)
                feature['title'] = track_data['name']
                feature['all_artists'] = ', '.join([artist['name'] for artist in track_data['artists']])
                feature['popularity'] = track_data['popularity']
                feature['release_date'] = track_data['album']['release_date']
                features_list.append(feature)
            print(f"Processed {index + 1}/{len(track_ids)} tracks.")  # Counter to track progress
            index += 1

    features_df = pd.DataFrame(features_list)
    return features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                        'danceability', 'energy', 'key', 'loudness', 'mode',
                        'acousticness', 'instrumentalness', 'liveness',
                        'valence', 'tempo', 'duration_ms', 'time_signature']]

# Example usage with unique_tracks obtained previously
unique_tracks_list = list(unique_tracks)  # Convert set to list if it's not already
features_df = get_track_features(unique_tracks_list)
print(features_df.head())

NameError: name 'unique_tracks' is not defined

In [ ]:
features_df.info() #This is the features df that I have created

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4630 entries, 0 to 4629
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                4630 non-null   object 
 1   title             4630 non-null   object 
 2   all_artists       4630 non-null   object 
 3   popularity        4630 non-null   int64  
 4   release_date      4630 non-null   object 
 5   danceability      4630 non-null   float64
 6   energy            4630 non-null   float64
 7   key               4630 non-null   float64
 8   loudness          4630 non-null   float64
 9   mode              4630 non-null   float64
 10  acousticness      4630 non-null   float64
 11  instrumentalness  4630 non-null   float64
 12  liveness          4630 non-null   float64
 13  valence           4630 non-null   float64
 14  tempo             4630 non-null   float64
 15  duration_ms       4630 non-null   float64
 16  time_signature    4630 non-null   float64


In [ ]:
def get_playlist_tracks(sp, playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    while results:
        tracks.extend([item['track'] for item in results['items'] if item['track']])
        results = sp.next(results) if results['next'] else None
    return tracks


def get_track_details_and_features(sp, tracks):
    track_ids = [track['id'] for track in tracks if track['id']]
    features_list = sp.audio_features(track_ids)

    details_list = []
    for track, features in zip(tracks, features_list):
        if features:  # Ensure there are features
            details = {
                'id': track['id'],
                'title': track['name'],
                'all_artists': ', '.join([artist['name'] for artist in track['artists']]),
                'popularity': track['popularity'],
                'release_date': track['album']['release_date'],
                'danceability': features['danceability'],
                'energy': features['energy'],
                'key': features['key'],
                'loudness': features['loudness'],
                'mode': features['mode'],
                'acousticness': features['acousticness'],
                'instrumentalness': features['instrumentalness'],
                'liveness': features['liveness'],
                'valence': features['valence'],
                'tempo': features['tempo'],
                'duration_ms': features['duration_ms'],
                'time_signature': features['time_signature']
            }
            details_list.append(details)
    return details_list

billboard_playlist_id = '6UeSakyzhiEt4NB3UAd6NQ'  # Replace with the actual ID
billboard_tracks = get_playlist_tracks(sp, billboard_playlist_id)
track_details = get_track_details_and_features(sp, billboard_tracks)

# Convert the list of details to a DataFrame
billboard_features_df = pd.DataFrame(track_details)
billboard_features_df.head()

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,97,2024-05-04,0.898,0.472,1,-7.001,1,0.0107,0.000000,0.1410,0.214,101.061,274192,4
1,7fzHQizxTqy8wTXwlrgPQQ,MILLION DOLLAR BABY,Tommy Richman,97,2024-04-26,0.849,0.699,1,-5.106,0,0.0982,0.000215,0.0680,0.927,138.003,155152,4
2,77DRzu7ERs0TX3roZcre7Q,euphoria,Kendrick Lamar,94,2024-04-30,0.831,0.643,1,-5.002,1,0.0460,0.000000,0.0840,0.142,139.948,383639,4
3,6dODwocEuGzHAavXqTbwHv,Fortnight (feat. Post Malone),"Taylor Swift, Post Malone",92,2024-04-19,0.504,0.386,11,-10.976,1,0.5020,0.000015,0.0961,0.281,192.004,228965,4
4,2FQrifJ1N335Ljm3TjTVVf,A Bar Song (Tipsy),Shaboozey,96,2024-04-12,0.722,0.709,9,-4.950,1,0.0633,0.000000,0.0804,0.604,81.012,171292,4


In [23]:
features_df = details_df

In [24]:
from google.colab import files

uploaded = files.upload()

Saving billboard_features_df.csv to billboard_features_df.csv


In [26]:
billboard_features_df = pd.read_csv('billboard_features_df.csv')

In [39]:
billboard_features_df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,rating,predicted_like
0,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,97,2024-05-04,0.923686,0.475513,0.008264,0.988152,1.0,0.010795,0.000000,0.114484,0.222824,0.001870,-0.007163,0.16,0,0.652071
1,7fzHQizxTqy8wTXwlrgPQQ,MILLION DOLLAR BABY,Tommy Richman,97,2024-04-26,0.873284,0.707129,0.008264,0.989449,0.0,0.099178,0.000220,0.036650,0.965224,0.002553,-0.007163,0.16,0,0.604172
2,77DRzu7ERs0TX3roZcre7Q,euphoria,Kendrick Lamar,94,2024-04-30,0.854769,0.649990,0.008264,0.989520,1.0,0.046451,0.000000,0.053709,0.147855,0.002589,-0.007163,0.16,0,0.662956
3,6dODwocEuGzHAavXqTbwHv,Fortnight (feat. Post Malone),"Taylor Swift, Post Malone",92,2024-04-19,0.518416,0.387765,0.090909,0.985430,1.0,0.507052,0.000016,0.066611,0.292587,0.003553,-0.007163,0.16,0,0.653774
4,2FQrifJ1N335Ljm3TjTVVf,A Bar Song (Tipsy),Shaboozey,96,2024-04-12,0.742652,0.717332,0.074380,0.989556,1.0,0.063926,0.000000,0.049871,0.628905,0.001499,-0.007163,0.16,0,0.597441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,47Xj8eIAp7hYOOqArmkqZE,Spin You Around (1/24),Morgan Wallen,81,2024-01-26,0.639791,0.446944,0.033058,0.988796,1.0,0.803008,0.000000,0.126213,0.627864,0.002627,-0.007163,0.16,0,0.620847
96,7CyPwkp0oE8Ro9Dd5CUDjW,"One Of The Girls (with JENNIE, Lily Rose Depp)","The Weeknd, JENNIE, Lily-Rose Depp",94,2023-06-23,0.271551,0.646929,0.066116,0.988686,1.0,0.039684,0.000002,0.125146,0.197834,0.001570,-0.007163,0.16,0,0.686218
97,55i5Hdv7VkcQtXCn1zyvKg,Let's Go - Remix,"Key Glock, Young Dolph",70,2024-04-12,0.722080,0.612238,0.066116,0.987955,1.0,0.001997,0.000000,0.371445,0.350896,0.001397,-0.007163,0.16,0,0.605620
98,0Jagf5iAYYiWAs0BEoWbWZ,Take Her Home,Kenny Chesney,63,2023-11-10,0.588361,0.835691,0.049587,0.989228,1.0,0.185844,0.000000,0.160332,0.539359,0.002757,-0.007163,0.16,0,0.592959


In [27]:
features_df['rating'] = 1

In [28]:
billboard_features_df['rating'] = 0

In [ ]:
features_df.to_csv('features_df.csv', index=False)
billboard_features_df.to_csv('billboard_features_df.csv', index=False)

from google.colab import files
files.download("features_df.csv")
files.download('billboard_features_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load DataFrames (assuming 'features_df' and 'billboard_features_df' are already loaded)
features = ['danceability', 'energy', 'key', 'loudness', 'mode',
            'acousticness', 'instrumentalness', 'liveness', 'valence',
            'tempo', 'duration_ms', 'time_signature']

# Normalize feature data
scaler = MinMaxScaler()
features_df[features] = scaler.fit_transform(features_df[features])
billboard_features_df[features] = scaler.transform(billboard_features_df[features])

# Prepare the target by binarizing the popularity (above median popularity)
median_popularity = features_df['popularity'].median()
features_df['target'] = (features_df['popularity'] > median_popularity).astype(int)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(features_df[features], features_df['target'], test_size=0.2, random_state=42)

In [40]:
X_train

,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
5777,0.797571,0.734295,1.000000,0.789717,0.0,0.000752,0.895431,0.050247,0.815494,0.598691,0.087436,0.8
4511,0.747976,0.648066,0.545455,0.838483,1.0,0.194775,0.002538,0.356466,0.781855,0.759448,0.042607,0.8
3780,0.707490,0.704215,0.818182,0.806554,0.0,0.519074,0.021929,0.101112,0.487258,0.874940,0.041517,0.8
5729,0.823887,0.815510,0.636364,0.840980,0.0,0.048991,0.003249,0.012047,0.857288,0.535978,0.035781,0.8
6218,0.704453,0.673132,0.545455,0.852757,0.0,0.131522,0.000000,0.364703,0.592253,0.625351,0.053687,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...
5226,0.452429,0.849601,0.181818,0.838332,1.0,0.033127,0.000076,0.655066,0.494393,0.769468,0.051107,0.8
5390,0.900810,0.455555,0.181818,0.751114,1.0,0.158630,0.000000,0.143328,0.914373,0.620870,0.043951,0.8
860,0.691296,0.676140,1.000000,0.874050,0.0,0.046682,0.000056,0.230848,0.139653,0.429179,0.054089,0.8
7603,0.810729,0.635031,0.636364,0.707925,1.0,0.012244,0.664975,0.105231,0.745158,0.562972,0.028006,0.8


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_dim=len(features)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
162/162 [==============================] - 1s 4ms/step - loss: 0.6899 - accuracy: 0.5349 - val_loss: 0.6822 - val_accuracy: 0.5710
Epoch 2/50
162/162 [==============================] - 0s 3ms/step - loss: 0.6758 - accuracy: 0.5619 - val_loss: 0.6708 - val_accuracy: 0.5610
Epoch 3/50
162/162 [==============================] - 0s 2ms/step - loss: 0.6733 - accuracy: 0.5747 - val_loss: 0.6687 - val_accuracy: 0.5880
Epoch 4/50
162/162 [==============================] - 0s 2ms/step - loss: 0.6677 - accuracy: 0.5855 - val_loss: 0.6666 - val_accuracy: 0.5988
Epoch 5/50
162/162 [==============================] - 0s 3ms/step - loss: 0.6647 - accuracy: 0.5855 - val_loss: 0.6650 - val_accuracy: 0.5972
Epoch 6/50
162/162 [==============================] - 0s 2ms/step - loss: 0.6648 - accuracy: 0.5957 - val_loss: 0.6631 - val_accuracy: 0.6042
Epoch 7/50
162/162 [==============================] - 0s 2ms/step - loss: 0.6628 - accuracy: 0.5966 - val_loss: 0.6610 - val_accuracy: 0.6142
Epoch 

In [43]:
billboard_features_df['predicted_like'] = model.predict(billboard_features_df[features])
top_recommendations = billboard_features_df.sort_values(by='predicted_like', ascending=False).head(10)
print(top_recommendations[['title', 'all_artists', 'predicted_like']])

4/4 [==============================] - 0s 5ms/step
                                             title  \
54                                     Made For Me   
26                                       Type Shit   
96  One Of The Girls (with JENNIE, Lily Rose Depp)   
43                                   Never Lose Me   
77                                      Cinderella   
52                                            loml   
8                                 Beautiful Things   
66                                   The Black Dog   
25                  Who’s Afraid of Little Old Me?   
37                   The Tortured Poets Department   

                                          all_artists  predicted_like  
54                                          Muni Long        0.693703  
26  Future, Metro Boomin, Travis Scott, Playboi Carti        0.688315  
96                 The Weeknd, JENNIE, Lily-Rose Depp        0.686218  
43                                          Flo Milli        0.683

In [42]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.6621
Test Accuracy: 0.6086
